In [24]:
from dotenv import load_dotenv
import openai
import os

In [25]:
from openai import OpenAI

In [26]:
# Load environment variables from the .env file
load_dotenv()

True

In [27]:
# Access the variables
api_key = os.getenv('OPENAI_API_KEY')

In [28]:
client = openai.OpenAI(api_key=api_key)

In [29]:
response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":"is it too late to join the course?"}]
)

In [30]:
response

ChatCompletion(id='chatcmpl-9c2b7dsOPsDWqUN5geq7RPRO7S1Rm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To determine whether it's too late to join a course, consider the following steps:\n\n1. **Check Enrollment Dates:** Look at the course catalog or the institution’s website to find key dates, such as enrollment deadlines.\n  \n2. **Contact the Instructor or Registrar:** Reach out directly to the course instructor or the institution's registrar office to ask if late enrollment is possible. Sometimes they can make exceptions.\n\n3. **Review Course Progress:** Determine how much of the course has already been completed. If only a small portion of the course has elapsed, catching up might be feasible.\n\n4. **Assess Personal Schedule:** Evaluate your own availability and ability to catch up on missed material if you are allowed to join late.\n\n5. **Understand the Implications:** Be aware that joining late might mean missing key in

In [31]:
response.choices[0].message.content

"To determine whether it's too late to join a course, consider the following steps:\n\n1. **Check Enrollment Dates:** Look at the course catalog or the institution’s website to find key dates, such as enrollment deadlines.\n  \n2. **Contact the Instructor or Registrar:** Reach out directly to the course instructor or the institution's registrar office to ask if late enrollment is possible. Sometimes they can make exceptions.\n\n3. **Review Course Progress:** Determine how much of the course has already been completed. If only a small portion of the course has elapsed, catching up might be feasible.\n\n4. **Assess Personal Schedule:** Evaluate your own availability and ability to catch up on missed material if you are allowed to join late.\n\n5. **Understand the Implications:** Be aware that joining late might mean missing key introductory material, which could impact your performance in the course.\n\nEach institution has its own policies regarding late enrollment, so it's best to get 

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

>Answer: 42f05b9372a9a4a470db3b52817899b99a76ee73

## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [27]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [28]:
from elasticsearch import Elasticsearch

In [29]:
es_client=Elasticsearch('http://localhost:9200')

In [30]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name='course-questions'

es_client.indices.create(index=index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [31]:
from tqdm.auto import tqdm

In [37]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: ProtocolError(('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))))